### In this NoteBook I try to generate fake samples from a  standard normal distribution from a noise distribution (Uniform Distribution) using generative adversarial networks.

In [0]:
import torch
import torch.utils.data as _data
#import matplotlib.pyplot as plt
from torch.distributions.normal import Normal
from torch.distributions.uniform import Uniform
m = Normal(torch.tensor([0.0]),torch.tensor([1.0]))
m1 = Uniform(torch.tensor([0.0]),torch.tensor([0.1]))
import torch.nn
import torch.optim
from torch.autograd.variable import Variable


In [3]:
data = []


for _ in range(100000):
        data.append(m.sample())


def noise(size):
    _data = []
    for _ in range(size):
        _data.append(m1.sample().item())

    temp = torch.Tensor(_data)
    if torch.cuda.is_available():
      return temp.view(-1,1).cuda()
    else:
      return temp.view(-1,1)


data = torch.Tensor(data)
data = _data.TensorDataset(data)
data_loader = _data.DataLoader(data,batch_size=1000,shuffle=True)
print(len(data_loader))

for n_batch,batch in enumerate(data_loader):
    #print(n_batch,batch[0].size(0))
    pass

d = batch[0].view(-1,1)


100


In [0]:
class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        n_features = 1
        n_out = 1
        self.hidden0 = torch.nn.Sequential(
                torch.nn.Linear(n_features,16),
                torch.nn.LeakyReLU(0.2),
                torch.nn.Dropout(0.2)
        )
        self.hidden1 = torch.nn.Sequential(
                torch.nn.Linear(16,4),
                torch.nn.LeakyReLU(0.2),
                torch.nn.Dropout(0.2)
        )
        self.hidden3 = torch.nn.Sequential(
                torch.nn.Linear(4,n_out),
                torch.nn.Sigmoid()
        )

    def forward(self,x):
        x = self.hidden0(x)
        #print("Inter 1",x.shape)
        x = self.hidden1(x)
        x = self.hidden3(x)
        return x


class GeneratorNet(torch.nn.Module):
    def __init__(self):
        super(GeneratorNet,self).__init__()
        n_features = 1
        n_out = 1

        self.hidden0 = torch.nn.Sequential(
                torch.nn.Linear(n_features,16),
                torch.nn.LeakyReLU(0.2)
                )

        self.hidden1 = torch.nn.Sequential(
                torch.nn.Linear(16,4),
                torch.nn.LeakyReLU(0.2)
                )
        self.hidden2 = torch.nn.Sequential(
                torch.nn.Linear(4,1),
                torch.nn.Tanh()
                )

    def forward(self,x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        return x




discriminator = Discriminator()
generator = GeneratorNet()


In [0]:
d_optimizer = torch.optim.Adam(discriminator.parameters(),lr=0.01)
g_optimizer = torch.optim.Adam(generator.parameters(),lr=0.01)

loss = torch.nn.BCELoss()


if torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()



def real_target(size):
    data = Variable(torch.ones(size,1))
    if torch.cuda.is_available():
      return data.cuda()
    else:
      return data

def zeros_target(size):
    data = Variable(torch.zeros(size,1))
    if torch.cuda.is_available():
      return data.cuda()
    else:
      return data


In [0]:
def train_discriminator(opt,real_data,fake_data):
    N = real_data.size(0)
    opt.zero_grad()
    pred_real = discriminator(real_data)
    error_r = loss(pred_real,real_target(N))
    error_r.backward()
    pred_fake = discriminator(fake_data)
    error_f = loss(pred_fake,zeros_target(N))
    opt.step()
    return error_r+error_f,pred_real,pred_fake

def train_generator(opt,fake_data):
    N = fake_data.size(0)
    opt.zero_grad()
    pred = discriminator(fake_data)
    error = loss(pred,real_target(N))
    error.backward()
    opt.step()
    return error



In [0]:
def main():

 num_epochs = 1000

 for epoch in range(num_epochs):
    for n_batch,batch in enumerate(data_loader):
        d = batch[0].view(-1,1)
        N = d.size(0)
        real_data = Variable(d)
        if torch.cuda.is_available():
          real_data = real_data.cuda()
        fake_data = generator(noise(N))
        d_er,d_pred_rl,p_pred_fk = train_discriminator(d_optimizer,real_data,fake_data)
        fake_data = generator(noise(N))
        g_error = train_generator(g_optimizer,fake_data)
    if epoch%50 == 0:
        fake_data = generator(noise(N))
        #print(epoch,fake_data.shape)
        print(epoch,torch.mean(fake_data,0),torch.std(fake_data,0),torch.mean(d,0),torch.std(d,0))


In [0]:
main()

0 tensor([0.9973], grad_fn=<MeanBackward2>) tensor([0.0001], grad_fn=<StdBackward1>) tensor([-0.0357]) tensor([1.0031])
